# Running the production TPS simulation

This is file runs the main calculation for the flexible length TPS simulation. It requires the file `alanine_dipeptide_tps_equil.nc`, which is written in the notebook `AD_tps_1_trajectory.ipynb`.

In this file, you will learn:

* how to load simulation objects from a file and reuse them
* how to run a production simulation

NB: This is a long calculation. In practice, it would be best to export the Python from this notebook, remove the `live_visualizer`, and run non-interactively on a computing node.

In [1]:
%matplotlib inline
import openpathsampling as paths

## Load simulation objects from file

In setting up the equilibration simulation, we've already defined everything we need for path sampling. One of the big strengths of OPS is that all simulation objects are saved in the storage file. This means that you can easily reload them for other simulations, and you have a clear chain of provenance, so you know that settings are *exactly* the same. This is why we name OPS objects using the `.named()` method -- it makes it easy to load them up later.

In this example, we'll create a new scheme. This scheme is actually identical to our equilibration scheme, so we could have just reused that old one. However, in many situations, you might use a different move scheme for equilibration than for production. For example, you might use only one-way shooting to equilibrate a TIS network, and but then use a full RETIS move scheme for the production run.

In [2]:
old_storage = paths.Storage("alanine_dipeptide_tps_equil.nc", mode='r')

In [3]:
network = old_storage.networks['tps_network']
engine = old_storage.engines['300K']
last_result = old_storage.steps[-1].active

In [4]:
# note that we could have loaded other things from storage, for example:
#C_7eq = old_storage.volumes['C_7eq']
#alpha_R = old_storage.volumes['alpha_R']
# however, we don't need to, since all the information is in the network

## Run TPS

As always, the process for setting up a simulation is:

1. Create a `network`
2. Create a `move_scheme`
3. Set up `initial_conditions`
4. Create the `PathSampling` object and run it.

Since we already created all the input to these when we set up the first trajectory, we can load use the versions we loaded above.

In [5]:
scheme = paths.OneWayShootingMoveScheme(network, 
                                        selector=paths.UniformSelector(),
                                        engine=engine)

In [6]:
initial_conditions = scheme.initial_conditions_from_trajectories(last_result)

No missing ensembles.
No extra ensembles.


In [7]:
storage = paths.Storage(#"tps_nc_files/
                        "alanine_dipeptide_tps.nc", "w")
storage.save(initial_conditions);  # save these to give storage a template

In [8]:
# note that we can only close the old storage after we've saved the initial
# conditions -- this is because details of the snapshot aren't loaded until
# needed
old_storage.close()

In [9]:
sampler = paths.PathSampling(storage=storage,
                             move_scheme=scheme,
                             sample_set=initial_conditions).named("Flexible_TPS_Sampling")

Note: 10000 steps will take a long time. If you just want to run a little bit, reduce this number.

In [10]:
sampler.run(100)#00)

Working on Monte Carlo cycle number 100
Running for 1 minute 20 seconds -  0.81 seconds per step
Estimated time remaining: 0 seconds
DONE! Completed 100 Monte Carlo cycles.


With this done, you can go on to do the flexible-length parts of the analysis in `AD_tps_3a_analysis_flex.ipynb`.